### LIBRARY IMPORT

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from contextlib import contextmanager
import time  
from sklearn.neighbors import NearestNeighbors  
from sklearn.preprocessing import minmax_scale  
from typing import Dict, List, Optional, Tuple  
import seaborn as sns 
import gc
import traceback 

import matplotlib.pyplot as plt

from scipy.stats import kendalltau

import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'  ## 서로 다른 환경에서는 이곳을 수정해야 함.
# working_directory = 'C:\\Users\\user\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'

os.chdir(working_directory)
gc.collect()



0

### Drop Early morning period & CHECK NULL VALUE

In [ ]:
# combined_result_df = pd.read_csv("./DB/professor_BTC_sum_both_10m.csv")
combined_result_df_raw = pd.read_csv("./output/BTC_sum_both_10m.csv")
print("# of rows of combined_Result_Df:", combined_result_df_raw.shape[0])

combined_result_df_raw['window_start'] = pd.to_datetime(combined_result_df_raw['window_start'])  # Convert to datetime

# Define the time range
start_time = pd.to_datetime('00:00:00').time()
end_time = pd.to_datetime('06:00:00').time()

# Filter and drop rows (새벽시간 삭제하기)
filtered_df = combined_result_df_raw[~combined_result_df_raw['window_start'].apply(lambda x: start_time <= x.time() <= end_time)]
print("# of rows of filtered_df:", filtered_df.shape[0])

# Set the target variable:
target_var = 'dv5_realized_volatility_mean0'
target_var_3 = target_var[:3]
print("target_var_3:", target_var_3)

# 사용하지 않을 변수(column)들 미리 제거 (dropna 효과를 최소화하기 위해.)
remove_varlist0 = [
                  'window_start', 
                  'window_end',
                  'prices_30s_for_NN',
                  'window_end_150_ticker',
                  'window_end_300_ticker',
                  'window_end_450_ticker',
                  'window_end_150_orderbook',
                  'window_end_300_orderbook',
                  'window_end_450_orderbook',
                  'volume_power',
                  'volume_power_150',
                  'volume_power_300',
                  'volume_power_450',
                  'end_price',

                #   'dv1_realized_volatility',
                  'dv2_lowest_return',
                  'dv3_highest_return',
                  'dv4_realized_volatility_30s',
                  'dv5_realized_volatility_mean0',

                  'prices_30s_for_NN_onlyprices',
                  ]
remove_varlist0.remove(target_var)

main_feature_list = list(filtered_df.columns)

for i in range(len(remove_varlist0)):
    # print(remove_varlist0[i])
    main_feature_list.remove(remove_varlist0[i])

filtered_df = filtered_df[main_feature_list]

combined_result_df = filtered_df.dropna()
print("# of rows of filtered_df:", combined_result_df.shape[0])

# 평균이 아닌 이전 값으로 Null 채우기
# combined_result_df = combined_result_df.fillna(method='ffill') 

combined_result_df.head(3)

### Manipulating Features => Log transformation of some liquidity measures (Why?)

In [ ]:
combined_result_df['liq_last_1'] = np.log10(combined_result_df['liq_last_1'] + 0.00001)
combined_result_df['liq_last_2'] = np.log10(combined_result_df['liq_last_2'] + 0.00001)
combined_result_df['liq_last_5'] = np.log10(combined_result_df['liq_last_5'] + 0.00001)
combined_result_df['liq_last_10'] = np.log10(combined_result_df['liq_last_10'] + 0.00001)
combined_result_df['liq_last_15'] = np.log10(combined_result_df['liq_last_15'] + 0.00001)
combined_result_df['trade_vol'] = np.log10(combined_result_df['trade_vol'] + 0.00001)
combined_result_df['num_trades'] = np.log10(combined_result_df['num_trades'] + 0.00001)

### NUMERIC FEATURES & CALCULATE CORR 

In [4]:
main_feature_list = list(combined_result_df.columns)
feature_list_for_corr = list(combined_result_df.columns)

remove_varlist = [
                  'time_id'
                  ]

for i in range(len(remove_varlist)):
    # print(remove_varlist[i])
    feature_list_for_corr.remove(remove_varlist[i])




In [ ]:
data = combined_result_df
correlation_matrix = data[feature_list_for_corr].corr(method=lambda x, y: kendalltau(x, y).correlation)
correlation_matrix_series = correlation_matrix[target_var].copy() # .sort_values(ascending=False)
correlation_matrix_series.sort_values(ascending=False, inplace=True)
correlation_matrix_series

In [7]:
combined_result_df_mfl = combined_result_df

In [14]:
feature_list_for_finding_NN = []
feature_list_for_finding_NN.append('realized_volatility_mean0')
feature_list_for_finding_NN.append('bidask_spread_0')
feature_list_for_finding_NN.append('bidask_spread_1')
feature_list_for_finding_NN.append('highest_return')
feature_list_for_finding_NN.append('lowest_return')
feature_list_for_finding_NN.append('num_trades')
feature_list_for_finding_NN.append('high_low_gap')
feature_list_for_finding_NN.append('BB_width_w10')
feature_list_for_finding_NN.append('BB_width_w20')
feature_list_for_finding_NN.append('liq_last_1')
feature_list_for_finding_NN.append('liq_last_5')
feature_list_for_finding_NN.append('ep_liq_1')
feature_list_for_finding_NN.append('ep_liq_5')
feature_list_for_finding_NN.append('tvpl1')
feature_list_for_finding_NN.append('tvpl5')
feature_list_for_finding_NN.append('tvpl_ep1')
feature_list_for_finding_NN.append('tvpl_ep5')
feature_list_for_finding_NN.append('trade_vol')
feature_list_for_finding_NN.append('trade.tau')
# feature_list_for_finding_NN.append('volume_power')


### BUILD NEIGHBORS

In [15]:
N_NEIGHBORS_MAX = 65 

class Neighbors:
    def __init__(self, 
                 name: str, 
                 pivot: pd.DataFrame, 
                 p: float, 
                 metric: str = 'minkowski', 
                 metric_params: object = None, 
                 exclude_self: bool = True,
                 ):
        self.name = name
        self.exclude_self = exclude_self
        self.p = p
        self.metric = metric
        self.neighbors = np.empty((0, 65), dtype=int)  # 빈 2D 배열로 초기화

        nn = NearestNeighbors(
            n_neighbors=N_NEIGHBORS_MAX, 
            p=p, 
            metric=metric, 
            metric_params=metric_params
        )
        
        # nn.fit(pivot)
        # _, self.neighbors = nn.kneighbors(pivot, return_distance=True)

        # 이웃을 찾을 이전 window 벙뮈
        window_size = 10000
        col_names = pivot.columns
        index_name = pivot.index.name
        
        for t in range(len(pivot)) :
            # window size 까지의 데이터는 random NN 설정
            # 1 ~ window 범위의 데이터는 추후 버려야 함
            if t < window_size :
                update_array = np.random.permutation(np.arange(1, 66))
                self.neighbors = np.append(self.neighbors, [update_array], axis = 0)
                
            else :
                pvdf = pd.DataFrame(pivot.iloc[t-window_size:t])
                pvdf.columns = [list(col_names)]
                pvdf = pvdf.rename_axis(index_name)
                nn.fit(pvdf)
                
                update_array = nn.kneighbors(pivot.iloc[t].values.reshape(1, -1), return_distance=False)
                update_array = update_array.reshape(-1)
                self.neighbors = np.append(self.neighbors, [update_array], axis = 0)
                # if t // 10000 == 0 :
                #    print(t,self.neighbors)

        self.columns = self.index = self.feature_values = self.feature_col = None

    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        raise NotImplementedError()

    def make_nn_feature(self, n=5, agg=np.mean) -> pd.DataFrame:
        assert self.feature_values is not None, "should call rearrange_feature_values beforehand"

        start = 1 if self.exclude_self else 0

        pivot_aggs = pd.DataFrame(
            agg(self.feature_values[start:n+start,:,0], axis=0), 
            columns=self.columns, 
            index=self.index
        )

        dst = pivot_aggs.reset_index() # unstack().
        # print("dst.shape:", dst.shape)
        new_column_names = ['time_id', f'{self.feature_col}_nn{n}_{self.name}_{agg.__name__}'] # 3개를 예측했는데 2개만 들어왔다??
        dst.columns = new_column_names 
        return dst
    

class TimeIdNeighbors(Neighbors):
    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        # feature_pivot = df.pivot(index='time_id', values=feature_col)
        # feature_pivot = feature_pivot.fillna(feature_pivot.mean())

        feature_df = df[['time_id', feature_col]]
        feature_df.set_index('time_id', inplace=True)
        feature_df = feature_df.fillna(feature_df.mean())

        feature_values = np.zeros((N_NEIGHBORS_MAX, feature_df.shape[0], 1))

        for i in range(N_NEIGHBORS_MAX):
            feature_values[i, :, 0] += feature_df.values[self.neighbors[:, i], 0]

        self.columns = list(feature_df.columns)
        self.index = list(feature_df.index)
        self.feature_values = feature_values
        self.feature_col = feature_col

    def __repr__(self) -> str:
        return f"time-id NN (name={self.name}, metric={self.metric}, p={self.p})"


### PROGRESS CHECK FUNCTION

In [16]:
@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}초')

def print_trace(name: str = ''):
    print(f'{name or "익명"}에서 에러가 발생했습니다.')
    print(traceback.format_exc())


### GET NN CLASS. Output: {}_sum_plus_nn_features.csv

In [ ]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

USE_ONE_FEATURE_C = True
USE_ONE_FEATURE_M_1 = True
USE_ONE_FEATURE_M_2 = True

USE_TWO_FEATURES = True

USE_ALL_FEATURES = True
USE_SEVALRAL_FEATURES = True

# Top 5 Related Feature
top_5_high_feat = list(correlation_matrix_series.keys())[1:6]
top_5_low_feat = list(correlation_matrix_series.keys())[-5:]

# Top 5 Absolute Related Feature
sorted_data = correlation_matrix_series.abs().sort_values(ascending=False)
top_5_high_abs_feat = list(sorted_data.head(6).keys())[1:]
top_5_low_abs_feat = list(sorted_data.tail(5).keys())

# time_id_neighbors List 
time_id_neighbors: List[Neighbors] = []

with timer('knn fit'):
    df_pv = combined_result_df_mfl.copy()
    # df_pv = df_pv.drop(['window_start', 'window_end','volume_power'], axis=1)
    
    # Standard All Feature
    # df_pv[feature_list_for_finding_NN] = scaler.fit_transform(df_pv[feature_list_for_finding_NN])

    feature_list = list(df_pv.columns)
    feature_list.remove('time_id') # Can't be standardized
    # feature_list.remove('prices_30s_for_NN_onlyprices') # Can't be standardized
    # feature_list.remove('time_id')
    df_pv[feature_list] = scaler.fit_transform(df_pv[feature_list])
    # combined_result_df_mfl_scaled.head(3)



    #### USE ONLY ONE FACTOR ####

    # ## Canberra Distance
    # if USE_ONE_FEATURE_C :
    #     for feat in feature_list_for_finding_NN :
    #         gc.collect()
    #         df_nn = df_pv[['time_id',feat]]
    #         df_nn.set_index('time_id', inplace=True)
    #         df_nn = df_nn.fillna(df_nn.mean())

    #         time_id_neighbors.append(
    #             TimeIdNeighbors(
    #                 feat + '_c', 
    #                 df_nn, 
    #                 p=2, 
    #                 metric='canberra', 
    #                 exclude_self=True
    #             )
    #         )

    ## Manhattan Distance
    
    if USE_ONE_FEATURE_M_1:
        for feat in feature_list_for_finding_NN :
            gc.collect()
            df_nn = df_pv[['time_id',feat]]
            df_nn.set_index('time_id', inplace=True)
            # df_nn = df_nn.fillna(df_nn.mean())

            time_id_neighbors.append(
                TimeIdNeighbors(feat + '_m_p1', df_nn, p=1, exclude_self=True)
            )

    # ## Euclidean Distance
    # if USE_ONE_FEATURE_M_2:
    #     for feat in feature_list_for_finding_NN :
    #         gc.collect()
    #         df_nn = df_pv[['time_id',feat]]
    #         df_nn.set_index('time_id', inplace=True)
    #         df_nn = df_nn.fillna(df_nn.mean())

    #         time_id_neighbors.append(
    #             TimeIdNeighbors(feat + '_m_p2', df_nn, p=2, exclude_self=True)
    #         )


    # #### TWO FACTOR ####

    # if USE_TWO_FEATURES:
    #     feature_list = ['time_id','realized_volatility_mean0','bidask_spread_0'] # target variable이 바뀌면 2번/3번 변수 수정해주어야.
    #     df_nn = df_pv[feature_list]
    #     df_nn.set_index('time_id', inplace=True)
    #     df_nn = df_nn.fillna(df_nn.mean())

    #     ## Canberra

    #     time_id_neighbors.append(
    #             TimeIdNeighbors(
    #                 'two_c',  # before correction: feat + 'two_c', 
    #                 df_nn, 
    #                 p=2, 
    #                 metric='canberra', 
    #                 exclude_self=True
    #             )
    #         )
    #     ## Euclidean Distance
    #     time_id_neighbors.append(
    #         TimeIdNeighbors(
    #             'two_m', 
    #             df_nn, 
    #             p=2, 
    #             exclude_self=True
    #         )
    #     )


    #### USE SEVALRAL FACTOR ####

    if USE_SEVALRAL_FEATURES:
        ## High Related Feature 
        feature_list = ['time_id']
        feature_list += top_5_high_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        # df_nn = df_nn.fillna(df_nn.mean())

        ### Euclidean Distance
        time_id_neighbors.append(
            TimeIdNeighbors(
                'high5_nn_m', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )

        ## Low Related Feature

        feature_list = ['time_id']
        feature_list += top_5_low_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        # df_nn = df_nn.fillna(df_nn.mean())        

        time_id_neighbors.append(
            TimeIdNeighbors(
                'low5_nn_m', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )

        ## High Abs Related Feature

        feature_list = ['time_id']
        feature_list += top_5_high_abs_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        # df_nn = df_nn.fillna(df_nn.mean())
        
        time_id_neighbors.append(
            TimeIdNeighbors(
                'high5_abs_nn_m', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )

        ## Low Abs Related Feature

        feature_list = ['time_id']
        feature_list += top_5_low_abs_feat
        df_nn = df_pv[feature_list]
        df_nn.set_index('time_id', inplace=True)
        # df_nn = df_nn.fillna(df_nn.mean())

        time_id_neighbors.append(
            TimeIdNeighbors(
                'low5_abs_nn_m', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )


    #### USE ALL FACTOR ####

    if USE_ALL_FEATURES:
        df_nn = df_pv.copy()
        # df_nn = df_nn.drop(['dv1_realized_volatility'], axis=1)
        df_nn.set_index('time_id', inplace=True)

        # Standard All Feature
        # df_nn[feature_list_for_finding_NN] = scaler.fit_transform(df_nn[feature_list_for_finding_NN])

        df_nn = df_nn[feature_list_for_finding_NN]
        # df_nn = df_nn.fillna(df_nn.mean())

        time_id_neighbors.append(
            TimeIdNeighbors(
                'all_nn_m_p1', 
                df_nn, 
                p=1, 
                exclude_self=True
            )
        )

        # time_id_neighbors.append(
        #     TimeIdNeighbors(
        #         'all_nn_m_p2', 
        #         df_nn, 
        #         p=2, 
        #         exclude_self=True
        #     )
        # )


#### Aggregate Features With NN

In [18]:
def make_nearest_neighbor_feature(df: pd.DataFrame) -> pd.DataFrame:
    df2 = combined_result_df_mfl.copy()
    print(df2.shape)

    ### time_id를 기준으로 얻어진 neighbor를 대상으로 feature 만들기
    feature_cols = {
        # 'realized_volatility': [np.mean], #, np.min, np.max, np.std
        'realized_volatility_mean0': [np.mean, np.median],
        'realized_volatility_30s': [np.mean, np.median],

        # 'dv1_realized_volatility': [np.mean],
        # 'dv2_lowest_return': [np.mean, np.median],
        # 'dv3_highest_return': [np.mean, np.median],
        # 'dv4_realized_volatility_30s': [np.mean, np.median],
        'dv5_realized_volatility_mean0': [np.mean, np.median],

        'num_trades': [np.mean, np.median],
        'lowest_return': [np.mean, np.median], # , np.mean, np.min
        'highest_return': [np.mean, np.median], # , np.mean, np.min
        'high_low_gap': [np.mean, np.median],
        'trade_vol': [np.mean, np.median],
        # 'volume_power': [np.mean, np.median],
        'BB_width_w10': [np.mean, np.median],
        'BB_width_w20': [np.mean, np.median],

        'liq_last_1': [np.mean, np.median],
        'liq_last_5': [np.mean, np.median],
        'ep_liq_1': [np.mean, np.median],
        'ep_liq_5': [np.mean, np.median],
        'bidask_spread_0': [np.mean, np.median],
        'bidask_spread_1': [np.mean, np.median],

        'tvpl1': [np.mean, np.median],
        'tvpl5': [np.mean, np.median],
        'tvpl_ep1': [np.mean, np.median],
        'tvpl_ep5': [np.mean, np.median],
        'trade.tau': [np.mean, np.median],
    }

    time_id_neigbor_sizes = [2, 4, 8, 16, 32, 48, 64] # 메모리 부족으로 계속 오류가 나는 것 같아 이웃 계산 숫자를 줄임.

    ndf: Optional[pd.DataFrame] = None
    
    # 새로운 feature를 기존 df에 추가하는 함수
    def _add_ndf(ndf: Optional[pd.DataFrame], dst: pd.DataFrame) -> pd.DataFrame:
        if ndf is None:
            return dst
        else:
            ndf[dst.columns[-1]] = dst[dst.columns[-1]].astype(np.float32)
            #columns_to_convert = [dst.columns[-1]]  # 열 변환 대상을 선택하거나 여러 열을 지정할 수 있음
            #converted_columns = dst[columns_to_convert].astype(np.float32)
            #ndf = pd.concat([ndf, converted_columns], axis=1)

            return ndf

    # neighbor time_id
    for feature_col in feature_cols.keys():
        gc.collect()
        try: 
            for nn in time_id_neighbors:
                nn.rearrange_feature_values(df2, feature_col)


            time_id_ns = time_id_neigbor_sizes

            for agg in feature_cols[feature_col]:
                for n in time_id_ns:
                    try:
                        for nn in time_id_neighbors:
                            gc.collect()
                            dst = nn.make_nn_feature(n, agg)
                            ndf = _add_ndf(ndf, dst)
                    except Exception:
                        WHERE_ERROR = feature_col
                        print_trace('time-id nn')
                        pass
        except Exception:
            print_trace('time-id nn')

    if ndf is not None:
        df2 = pd.merge(df2, ndf, on=['time_id'], how='left')
    
    print(df2.shape)

    return df2

In [ ]:
pd.set_option('display.max_columns', None)

combined_result_df.head(3)

In [20]:
gc.collect()

with timer('make nearest neighbor feature'):
    df3 = make_nearest_neighbor_feature(combined_result_df_mfl)

gc.collect()

(75169, 89)


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


(75169, 7145)
[make nearest neighbor feature]  548.499초


0

In [21]:
# Cut the first 'window_size' (10,000) rows. <= These rows do not have appropriate nearest neighbors.
df3 = df3.drop(index=range(window_size))

coin = 'BTC'
df3.to_csv(working_directory + "output\\{}_sum_plus_nn_features_for_{}.csv".format(coin, target_var_3), index=False)

In [ ]:
# TEMP code

import gc
import pandas as pd
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'  ## 서로 다른 환경에서는 이곳을 수정해야 함.
# working_directory = 'C:\\Users\\user\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'

os.chdir(working_directory)
gc.collect()

# combined_result_df = pd.read_csv("./DB/professor_BTC_sum_both_10m.csv")
coin = 'BTC'
target_var = 'dv5_realized_volatility_mean0'
target_var_3 = target_var[:3]
df3 = pd.read_csv("./output/{}_sum_plus_nn_features_for_{}.csv".format(coin, target_var_3))

window_size=10000
df3 = df3.drop(index=range(window_size))
df3.to_csv(working_directory + "output\\{}_sum_plus_nn_features_for_{}.csv".format(coin, target_var_3), index=False)


### Checking the final outcome

In [2]:
import gc
import pandas as pd
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'  ## 서로 다른 환경에서는 이곳을 수정해야 함.
# working_directory = 'C:\\Users\\user\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'

os.chdir(working_directory)
gc.collect()

# combined_result_df = pd.read_csv("./DB/professor_BTC_sum_both_10m.csv")
coin = 'BTC'
target_var = 'dv5_realized_volatility_mean0'
target_var_3 = target_var[:3]
df3 = pd.read_csv("./output/{}_sum_plus_nn_features_for_{}.csv".format(coin, target_var_3))

df3.head(3)
print("# of rows:", df3.shape[0])
print("# of columns:", df3.shape[1])

# of rows: 75169
# of columns: 7145


In [3]:
df3.columns

Index(['realized_volatility_mean0', 'num_trades', 'lowest_return',
       'highest_return', 'high_low_gap', 'trade_vol', 'time_id',
       'BB_width_w20', 'BB_width_w40', 'BB_width_w10',
       ...
       'trade.tau_nn64_tvpl5_m_p1_median',
       'trade.tau_nn64_tvpl_ep1_m_p1_median',
       'trade.tau_nn64_tvpl_ep5_m_p1_median',
       'trade.tau_nn64_trade_vol_m_p1_median',
       'trade.tau_nn64_trade.tau_m_p1_median',
       'trade.tau_nn64_high5_nn_m_median', 'trade.tau_nn64_low5_nn_m_median',
       'trade.tau_nn64_high5_abs_nn_m_median',
       'trade.tau_nn64_low5_abs_nn_m_median',
       'trade.tau_nn64_all_nn_m_p1_median'],
      dtype='object', length=7145)